In [2]:
d1 = "plot: two teen couples go to a church party, drink and then drive."
d2 = "films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before . "
d3 = "every now and then a movie comes along from a suspect studio , with every indication that it will be a stinker , and to everybody's surprise ( perhaps even the studio ) the film becomes a critical darling . "
d4 = "damn that y2k bug . "
documents = [d1, d2, d3, d4]

## Stemming

In [3]:
import nltk, string, numpy
nltk.download('punkt') # first-time use only

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
stemmer = nltk.stem.porter.PorterStemmer()
def StemTokens(tokens):
	return [stemmer.stem(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def StemNormalize(text):
	return StemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [8]:
nltk.download('wordnet') # first-time use only
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
	return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
	return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
LemVectorizer = CountVectorizer(tokenizer=LemNormalize, stop_words='english')
LemVectorizer.fit_transform(documents)

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<4x41 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [13]:
print(LemVectorizer.vocabulary_)

{'plot': 27, 'teen': 37, 'couple': 9, 'church': 6, 'party': 25, 'drink': 14, 'drive': 15, 'film': 17, 'adapted': 0, 'comic': 8, 'book': 3, 'plenty': 26, 'success': 32, 'theyre': 38, 'superheroes': 33, 'batman': 2, 'superman': 34, 'spawn': 29, 'geared': 18, 'kid': 22, 'casper': 5, 'arthouse': 1, 'crowd': 11, 'ghost': 19, 'world': 39, 'really': 28, 'like': 23, 'hell': 20, 'movie': 24, 'come': 7, 'suspect': 36, 'studio': 31, 'indication': 21, 'stinker': 30, 'everybodys': 16, 'surprise': 35, 'critical': 10, 'darling': 13, 'damn': 12, 'y2k': 40, 'bug': 4}


In [15]:
tf_matrix = LemVectorizer.transform(documents).toarray()
print(tf_matrix)

[[0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
  0 1 0 0 0]
 [1 1 1 2 0 1 0 0 2 0 0 1 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 0 1 1 1 0
  0 0 1 1 0]
 [0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 2 0 0 0 1
  1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1]]


In [16]:
tf_matrix.shape

(4, 41)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfTran = TfidfTransformer(norm="l2")
tfidfTran.fit(tf_matrix)
print(tfidfTran.idf_)

[1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.51082562
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.91629073 1.91629073]


#### Now we have a vector where each component is the idf for each term. In this case, the values are almost the same because other than one term, each term only appears in 1 document. The exception is the 18th term that appears in 2 document. We can corroborate the result.

In [20]:
import math
def idf(n,df):
    result = math.log((n+1.0)/(df+1.0)) + 1
    return result
print("The idf for terms that appear in one document: " + str(idf(4,1)))
print("The idf for terms that appear in two documents: " + str(idf(4,2)))

The idf for terms that appear in one document: 1.916290731874155
The idf for terms that appear in two documents: 1.5108256237659907


In [22]:
tfidf_matrix = tfidfTran.transform(tf_matrix)
print(tfidf_matrix.toarray())

[[0.         0.         0.         0.         0.         0.
  0.37796447 0.         0.         0.37796447 0.         0.
  0.         0.         0.37796447 0.37796447 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.37796447 0.         0.37796447 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.37796447 0.         0.         0.        ]
 [0.19381304 0.19381304 0.19381304 0.38762607 0.         0.19381304
  0.         0.         0.38762607 0.         0.         0.19381304
  0.         0.         0.         0.         0.         0.15280442
  0.19381304 0.19381304 0.19381304 0.         0.19381304 0.19381304
  0.         0.         0.19381304 0.         0.19381304 0.19381304
  0.         0.         0.19381304 0.19381304 0.19381304 0.
  0.         0.         0.19381304 0.19381304 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.27094807 0.         0.         0.27094807 0.
  0.

In [23]:
cos_similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()
print(cos_similarity_matrix)

[[1.         0.         0.         0.        ]
 [0.         1.         0.03264186 0.        ]
 [0.         0.03264186 1.         0.        ]
 [0.         0.         0.         1.        ]]


#### The matrix obtained in the last step is multiplied by its transpose. The result is the similarity matrix, which indicates that d2 and d3 are more similar to each other than any other pair.

## Use TfidfVectorizer instead:

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()
cos_similarity(documents)

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.03264186, 0.        ],
       [0.        , 0.03264186, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])